<a href="https://colab.research.google.com/github/mdeniz1/datascienceprojects/blob/main/turkishsentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a fine tuned [BERTurk](https://huggingface.co/dbmdz/bert-base-turkish-cased) model for  sentiment analysis in Turkish on product review [data set](https://huggingface.co/datasets/W4nkel/turkish-sentiment-dataset)

You can try out a demo of the model [here](https://huggingface.co/spaces/mdeniz1/turkish-sentiment-analysis)

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... do

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
raw_datasets = load_dataset("W4nkel/turkish-sentiment-dataset")
raw_datasets

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 4800
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 1200
    })
})

In [ ]:
import pandas as pd
raw_datasets.set_format(type="pandas")
df = raw_datasets["train"][:]


In [ ]:
df.head(10)

,label,text
0,0,kargo süresi cok uzun gün makina fena değil
1,0,ürün demonte geldi fakat kurulum şemasi anlaşi...
2,0,allah seni ve senin gibi düşünen mahlukların c...
3,2,ürün ev kullanımı için çok iyibir kere oldukça...
4,2,ürün gayet başarılı teşekkür ederim hepsiburad...
5,2,arkadaslar öncelikle belirteyim ki ben bir tv ...
6,2,Konu Kurgu efekt oyunculuk her şey gayet iyiyd...
7,0,arkadaslar gunduz sikinti yokgece cocuk sirils...
8,1,Bu yatırımlar hala devam etmektedir
9,1,Aynı zamanda bu teknolojiyi yurt dışına da satar


In [ ]:
raw_datasets["train"].features

{'label': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None)}

The label type is int64 instead of ClassLabel. Therefore, I conducted a manual inspection of the labels and determined that 0 corresponds to a negative product review, 1 signifies a neutral product review, and 2 denotes a positive review.

In [ ]:
#raw_datasets_2 = load_dataset("winvoker/turkish-sentiment-analysis-dataset")
#raw_datasets_2

In [ ]:
#raw_datasets_2['train'].features

In [ ]:
#raw_datasets_2.set_format(type="pandas")
#df2 = raw_datasets_2["train"][:]
#df2.head(20)

In [ ]:

#df2 = df2.drop(columns=['dataset'])

#label_mapping = {'Positive': 2, 'Negative': 0, 'Notr': 1}
#df2['label'] = df2['label'].map(label_mapping)
#df2.head()

In [ ]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'label': 0, 'text': 'kargo süresi cok uzun gün makina fena değil'}

In [ ]:
raw_train_dataset[:6]

{'label': [0, 0, 0, 2, 2, 2],
 'text': ['kargo süresi cok uzun gün makina fena değil',
  'ürün demonte geldi fakat kurulum şemasi anlaşilir değilen azindan numaralandirilmis olarak parcalar gelse daha bir anlamli olur ayaklarin nereye takilacagi belli degilurunun bazi parcalarinin kenarlarinda deformasyon varalirken biraz daha arastirin derim',
  'allah seni ve senin gibi düşünen mahlukların cezasını versin inşlah',
  'ürün ev kullanımı için çok iyibir kere oldukça hafif olması kolu yormuyordarbeli çalışma modu süperduvar delerken çok başarılıkesinlikle tavsiye ederim',
  'ürün gayet başarılı teşekkür ederim hepsiburadacom',
  'arkadaslar öncelikle belirteyim ki ben bir tv tamircisi olarak montaj yapmaktayım bu fiyata böyle bir tv daha önce hiç görmedim ve kendim de bu tv den aldımfullhd uydu alıcısı kullanmak şartıyla mükemmel bir görüntüsü varbu fiyata kacırılmayacak kadar cazip bir tvherkese tavsiye edıyorum']}

In [ ]:
raw_train_dataset.features

{'label': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None)}

In [ ]:
from transformers import AutoTokenizer

checkpoint = "dbmdz/bert-base-turkish-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/4800 [00:00<?, ? examples/s]

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4800
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1200
    })
})

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_datasets["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [ ]:
import tensorflow as tf

#model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics="accuracy")

In [ ]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3
600/600 [==============================] - 285s 385ms/step - loss: 0.2851 - accuracy: 0.8894 - val_loss: 0.1868 - val_accuracy: 0.9358
Epoch 2/3
600/600 [==============================] - 157s 261ms/step - loss: 0.1193 - accuracy: 0.9598 - val_loss: 0.1933 - val_accuracy: 0.9392
Epoch 3/3
600/600 [==============================] - 160s 267ms/step - loss: 0.0373 - accuracy: 0.9904 - val_loss: 0.2354 - val_accuracy: 0.9325


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
model.save('/content/drive/My Drive/Colab Notebooks/turkish_sentiment')

In [ ]:
import tensorflow as tf


In [ ]:
model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/turkish_sentiment')


In [ ]:
from transformers import pipeline

model = pipeline("text-classification", model="mdeniz1/turkish-sentiment-analysis-bert-base-turkish-uncased")


def predict(prompt):
    sentiment=model(prompt)
    if sentiment[0]['label']=='LABEL_2':
        return 'positive'
    elif sentiment[0]['label']=='LABEL_1':
        return 'neutral'
    else:
        return 'negative'

Some layers from the model checkpoint at mdeniz1/turkish-sentiment-analysis-bert-base-turkish-uncased were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at mdeniz1/turkish-sentiment-analysis-bert-base-turkish-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further

In [ ]:
model('muhteşem bir ürün')[0]['label']

'LABEL_2'

In [ ]:
predict('muhteşem bir ürün')

'positive'

In [ ]:
import gradio as gr

title = "Turkish Sentiment Analysis"
description = """
The bot has been trained to analyze the sentiment of Turkish text. When provided with a Turkish prompt, it can determine whether the text expresses a neutral, negative, or positive sentiment.

"""
textbox = gr.Textbox(label="Enter a text here:", lines=1)

gr.Interface(
    fn=predict,
    inputs=textbox,
    outputs="text",
    title=title,
    description=description,
    examples=[["ortam oldukça güzel"], ["senden bir cacık olmaz"]],
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9891ab229ef367c593.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
